In [31]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import sys
from torchvision.transforms import v2
import torch

sys.path.append('..')

from utils import load_csv
from preprocessing import tiff_handling

In [32]:
frame = load_csv.load_pandas()

# get me 2 samples of inliers and outliers
inliers = load_csv.get_correct_data(frame)
inliers = inliers.sample(2)
outliers = load_csv.get_outliers(frame)
outliers = outliers.sample(2)

transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((60, 80)),
        v2.ToTensor(),
        v2.Lambda(
            lambda x: (x.view(-1) - torch.min(x)) / (torch.max(x) - torch.min(x))
        ),
    ]
)

/home/bendm/machine_learning/project/majority-vote-outlier-detection/plots_and_images/../utils/load_csv.py:20: DtypeWarning:

Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.

/home/bendm/machine_learning/project/majority-vote-outlier-detection/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning:

The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.



In [33]:
inliers["image"] = inliers["file_names"].apply(lambda x: transform(tiff_handling.handle_tiff(x))).sample(2)
outliers["image"] = outliers["file_names"].apply(lambda x: transform(tiff_handling.handle_tiff(x))).sample(2)

## plot out the images

In [34]:
# plot the images using plotly
import plotly.express as px

inliers = inliers["image"].values
outliers = outliers["image"].values


# plot the images using plotly express
fig = px.imshow(inliers[0].view(60, 80))
fig.show()
fig.write_image("inlier1.png")
fig = px.imshow(inliers[1].view(60, 80))
fig.show()
fig.write_image("inlier2.png")
fig = px.imshow(outliers[0].view(60, 80))
fig.show()
fig.write_image("outlier1.png")
fig = px.imshow(outliers[1].view(60, 80))
fig.show()
fig.write_image("outlier2.png")

# visualize inlier and outlier reconstructions